In [1]:
import numpy as np
import pandas as pd
import io

In [2]:
from google.colab import files
uploaded = files.upload()


Saving ufc_event_data_dates_modded.csv to ufc_event_data_dates_modded.csv
Saving ufc-master-unmodified-dates-modded.csv to ufc-master-unmodified-dates-modded.csv
Saving ufc_fight_data.csv to ufc_fight_data.csv
Saving ufc_fight_stat_data.csv to ufc_fight_stat_data.csv
Saving ufc_fighter_data.csv to ufc_fighter_data.csv


In [ ]:
# print(uploaded.keys())

ufc_fight_data = pd.read_csv(io.BytesIO(uploaded['ufc_fight_data.csv']))
ufc_fight_stat_data = pd.read_csv(io.BytesIO(uploaded['ufc_fight_stat_data.csv']))
ufc_fighter_data = pd.read_csv(io.BytesIO(uploaded['ufc_fighter_data.csv']))
# ufc_event_data = pd.read_csv(io.BytesIO(uploaded['ufc_event_data.csv']))
ufc_event_data = pd.read_csv(io.BytesIO(uploaded['ufc_event_data_modded.csv']))
# ufc_old_data = pd.read_csv(io.BytesIO(uploaded['ufc-master-unmodified.csv']))
ufc_old_data = pd.read_csv(io.BytesIO(uploaded['ufc-master-unmodified-modded.csv']))

#for printing an example of each data set
# ufc_fight_data_example = pd.read_csv(io.BytesIO(uploaded['ufc_fight_data.csv'])).head(8)
# ufc_fight_stat_data_example = pd.read_csv(io.BytesIO(uploaded['ufc_fight_stat_data.csv'])).head(8)
# ufc_fighter_data_example = pd.read_csv(io.BytesIO(uploaded['ufc_fighter_data.csv'])).head(8)
# ufc_event_data_example = pd.read_csv(io.BytesIO(uploaded['ufc_event_data.csv'])).head(8)

# ufc_fight_data_example
# ufc_fight_stat_data_example
# ufc_fighter_data_example
# ufc_event_data_example

KeyError: 'ufc_event_data_modded.csv'

In [ ]:
#Merge all new data sets together

# merge ufc_fighter_data with ufc_fight_stat_data on fighter_id
merged_data = ufc_fighter_data.merge(ufc_fight_stat_data, on='fighter_id', how='inner')

# 1. Drop the 'fighter_url' column from ufc_fight_data. Prevents duplicate columns error. only needs to be ran once per session.
# ufc_fight_data.drop('fight_url', axis=1, inplace=True)

# 2. Merge 'merged_data' and 'ufc_fight_data' on 'fight_id'
result_df = merged_data.merge(ufc_fight_data, on='fight_id', how='inner')



final_df = result_df.merge(ufc_event_data, on='event_id', how='inner')


# final_df


In [ ]:
old_and_new_df = final_df


In [ ]:
# prompt: display every row in final_df that has a fighter_f_name of Jon and a fighter_l_name of Jones. Do it in a way so it displays every column

final_df[(final_df['fighter_f_name'] == 'Jon') & (final_df['fighter_l_name'] == 'Jones')]


In [ ]:
# prompt: print the column names of final_df and ufc_old_data

# final_df
# final_df[
#     (final_df['fighter_f_name'] == 'Jon') &
#     (final_df['fighter_l_name'] == 'Jones')
# ]


final_df[(final_df['fighter_f_name'] == 'Jon') & (final_df['fighter_l_name'] == 'Jones')]

# print(final_df.columns)
# print(ufc_old_data.columns)


In [ ]:
data_transfer_column_names = [
    'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw',
    'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission', 'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw',
    'R_odds', 'B_odds', 'r_dec_odds', 'b_dec_odds', 'r_sub_odds', 'b_sub_odds', 'r_ko_odds', 'b_ko_odds'
]


ufc_old_data_reduced = ufc_old_data[['R_fighter', 'date'] + data_transfer_column_names]

# print(final_df['event_date'].dtype)  # Should be 'datetime64[ns]'
# print(ufc_old_data_reduced['date'].dtype)  # Should also be 'datetime64[ns]'
# print('--------------------')
# print(final_df[['fighter_f_name', 'fighter_l_name', 'event_date']].head())
# print(ufc_old_data_reduced[['R_fighter', 'date']].head())\

#event_date in final_df and date in ufc_old_data_reduced are currently string objects (dtype: object) rather than datetime objects. This causes the merge to fail, this is how we fix that.
final_df['event_date'] = pd.to_datetime(final_df['event_date'])
ufc_old_data_reduced['date'] = pd.to_datetime(ufc_old_data_reduced['date'])
# print(final_df['event_date'].head())
# print(ufc_old_data_reduced['date'].head())

'''
def merge_fight_data(fight_row, ufc_old_data_reduced):
    fighter_name = fight_row['fighter_f_name'] + " " + fight_row['fighter_l_name']  # Or your combination method
    fight_date = fight_row['event_date']

    match = ufc_old_data_reduced[(ufc_old_data_reduced['R_fighter'] == fighter_name) &
                                 (ufc_old_data_reduced['date'] == fight_date)]

    if not match.empty:
        return pd.concat([fight_row, match.iloc[0, 2:]], axis=1)  # Combine rows
    else:
        return fight_row  # No match found, return the original row

# Apply the merge to each row in final_df
final_df = final_df.apply(merge_fight_data, args=(ufc_old_data_reduced,), axis=1)
'''


ValueError: time data "8/21/2021" doesn't match format "%Y-%m-%d", at position 5. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.